# Chapter 1: Single cell simulation with current injection (with BioNet)

In this example we will build a network consisting of a single biophysically detailed cell. Then we will run a short simulation with a current injection at the soma of the cell, then look at the output of spikes, membrane potential and calcium flux.


**Note** - scripts and files for running this tutorial can be found in the directory [sources/chapter01/](sources/chapter01)

**requirements:**
* Python 2.7, 3.6+
* bmtk
* NEURON 7.4+

## 1. Building the network

First step is to use the bmtk Network Builder to create and save the network. First we instantiate a network with a name or our choosing (since throughout this tutorial we will use cell models that from the mouse cortex, let's call our network 'mcortex'). 

Once we have a network, we can add a single node by calling the add_nodes method.

In [1]:
from bmtk.builder.networks import NetworkBuilder

net = NetworkBuilder('mcortex')
net.add_nodes(cell_name='Scnn1a_473845048',
              potental='exc',
              model_type='biophysical',
              model_template='ctdb:Biophys1.hoc',
              model_processing='aibs_perisomatic',
              dynamics_params='472363762_fit.json',
              morphology='Scnn1a_473845048_m.swc')

/home/ben/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Some of the parameters used to create the node are optional and only for our benefit. Others are necessary for when we will eventually run a simulation:
* *cell_name* (optional) - Name/type of cell we will be modeling.
* *potential* (optional) - Use to indicate that it is an excitatory type cell.
* *model_type* - Used by the simulator to indicate that we are using a biophysical cell.
* *dynamics_params* - Model parameters. File will be downloaded from the Allen Cell Types Database. 
* *morphology* - Model morphology. File will be downloaded from the Allen Cell Types Database.
* *model_processing* - A custom function used by the simulator to load the model into NEURON using Allen Cell-Types files for perisomatic models
 
#### Building and saving
The final thing to do is to build and save the network. If successful, we should see a combination of hdf5 and csv files in the './network' directory, these files are used describe the network, and can be saved, stored and run at a later date.

First, it's a good idea to remove any old files in the "network" folder so they don't interfere with the current simulation.

In [2]:
%%bash
rm network/*

If you get the output:

rm: cannot remove 'network/*': No such file or directory

It's OK. Keep going.

In [3]:
net.build()
net.save_nodes(output_dir='network')

Use the NetworkBuilder nodes() method to show that a node of our parameters was created

In [4]:
from pprint import pprint
for node in net.nodes():
    pprint(node)

{'node_type_id': 100, 'dynamics_params': '472363762_fit.json', 'potental': 'exc', 'morphology': 'Scnn1a_473845048_m.swc', 'node_id': 0, 'model_template': 'ctdb:Biophys1.hoc', 'model_type': 'biophysical', 'model_processing': 'aibs_perisomatic', 'cell_name': 'Scnn1a_473845048'}


## 2. Setting up the simulator enviornment

Now that the network has been built, we can use the BioNet simulator to setup and run it using NEURON. We will need to set-up the paths to the configuration and parameters files. We can do this by either:
1. Copying the directory tree from LOCATION into the current directory. Then compiling the extra NEURON mechanisms used by the Allen Cell Models by running the command 
```bash
$ cd components/mechanism
$ nrnivmodl modfiles
```
2. We can build the enviornment by scratch, which we can do from a command-line
```bash
$ python -m bmtk.utils.sim_setup -n network --membrane_report-vars v,cai --membrane_report-sections soma --tstop 2000.0 --dt 0.1 bionet
```
3. Call the function directly in python

In [5]:
from bmtk.utils.sim_setup import build_env_bionet
build_env_bionet(network_dir='network', tstop=2000.0, dt=0.1,
                 reports={'membrane_report': {
                         "module": "membrane_report",
                         'cells': 'all',
                         'variable_name': ['cai', 'v'],
                         "file_name": "cell_vars.h5",
                         "sections": "soma",
                     }})

/home/ben/bmtk/docs/tutorial/biophys_components/mechanisms
Was unable to compile mechanism in $COMPONENTS_DIR/mechanisms


This creates standard SONATA configuration files for BioNet, using the built network files in ./network and having a simulation run-time of 2 seconds. The membrane_report-vars and membrane_report-sections sets it so that we are recording membrane potential (v) and calcium concentration (cai) at the soma.:

#### Set the model files
The cell will be modeled from a V1, layer 4 Scnn1a mouse cell. Download the model file from http://celltypes.brain-map.org/neuronal_model/download/482934212, or by searching the Allen Cell-Types Database. We will need to unzip the downloaded package and move the parameters and morphology files into our simulator enviornment:
* Rename *fit_parameters.json* to *472363762_fit.json* and place in ./components/biophysical/electrophysiology/
* Rename *reconstruction.swc* to *Scnn1a_473845048_m.swc* and place in ./components/biophysical/morphologies/ 

In [6]:
%%bash
cp sources/bionet_files/components/biophysical/electrophysiology/472363762_fit.json biophys_components/biophysical_neuron_templates/
cp sources/bionet_files/components/biophysical/morphology/Scnn1a_473845048_m.swc biophys_components/morphologies/

#### Checking the simulation_config.json

The file ./simulation_config.json is used to set network, run-time and output parameters during a simulation. In the prefered bmtk workflow the config file is used to update and adjust parameters on different simulations runs, eliminating the need for complicated programs and allows modelers and experimentalist to run their models with little-to-no programming required.

More about the configuration file will be explained throughout the tutorials. However current what is missing is any input stimulus. Without it the simulation will run, but not do anything interesting. To add a current injection into the soma of our cell, we must open simulation_config.json with a text editor and modify the **input** section to:
```json
"inputs": {
 	"current_clamp": {
      "input_type": "current_clamp",
      "module": "IClamp",
      "node_set": "all",
      "amp": 0.120,
      "delay": 500.0,
      "duration": 1000.0
    }
},
```

This will inject a current (amp) of .12 mV with a delay of 500 ms and a duration of 1000 ms (i.e from times 500 - 1500 ms). 


## 3. Running the simulation

Once our config file is setup we can run a simulation either through the command line:
```bash
$ python run_bionet.py simulation_config.json
```

or through the script

In [7]:
from bmtk.simulator import bionet


conf = bionet.Config.from_json('simulation_config.json')
conf.build_env()
net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()

Failed to setup nrnpy_pr
2019-05-28 09:23:43,104 [INFO] Created log file
NEURON mechanisms not found in ./biophys_components/mechanisms.
2019-05-28 09:23:43,150 [INFO] Building cells.


ValueError: argument not a density mechanism name.

A quick breakdown of the script:
```python
conf = config.from_json('config.json')
io.setup_output_dir(conf)
nrn.load_neuron_modules(conf)
```
This section loads the configuration file, it setups the output directory and files for writing during the simulation, and loads NEURON mechanisms needed by the cell model(s) during the simulation.

```python
net = bionet.BioNetwork.from_config(conf)
```
Creates a NEURON representation of the network, including cell models that have been converted into their NEURON equivelents.

```python
sim = Simulation.from_config(conf, network=net)
sim.run()
```
Sets up and runs the NEURON simulation. When finished the output - spike times, membrane potential and Calcium influx, will be saved into the output directory as specified in the config.



## 4. Analyzing the run

The results of the simulation are placed into various files as specified in the "output" section of the config file. We can change this before run-time if required.

All simulations will save the spike times of the network cells. These are saved in csv format (output/spikes.txt) or hdf5 format(output/spikes.h5). To get the table of spike times for our single-cell network we can run the following method from the analyzer (gid 0 corresponds to our single cell).

In [ ]:
from bmtk.analyzer.spike_trains import to_dataframe
to_dataframe(config_file='simulation_config.json')

When setting up the enviornment and config file we specified cell_vars=['v', 'cai']. This indicates to the simulator to also record membrane potential and calcium diffusion (and we can also specify other variables as long as they are supported in NEURON). The recordings are stored in hdf5 format in the file output/cellvars/0.h5 (0 for the gid of our first and only cell).

In [ ]:
from bmtk.analyzer.cell_vars import plot_report

plot_report(config_file='simulation_config.json')

## 5. Additional Information

### Changing run-time parameters.
By making changes to the config file, we can change the conditions and simulation parameters without having to rebuild the network, modify parameter files, or change our run_bionet script. In fact we can iteratively run multiple simulations without any extra coding, only a text editor to change the json file.

The run section of the config.json contains most of the parameters unique to the simulation:
```json
"run": {
    "dL": 20,
    "nsteps_block": 5000, 
    "spike_threshold": -15,
    "tstop": 2000.0, 
    "dt": 0.1
}
```
* tstop - simulation runtime in milliseconds.
* dt - the time steps of the simulation. decreasing dt should increase accuracy of firing dynamics, but also increase time it takes to complete.
* spike_threshold - used to determine when to count a action potential
* dL - length of segments in a section. the number of segments will be forced to an odd number so the actual length may vary.

Through the conditions section we can adjust simulation temperature (C) and the initial membrane potential of the cells:
```json
"conditions": {
    "celsius": 34.0, 
    "v_init": -80
}
```

And lastly, the input section lets us control stimulus onto the network. There are a number of different options which will be explained in the following tutorials. But even with a simple current injection we can adjust amplitude, delay and stimulation duration and measure the effect on the cell.
```json
"inputs": {
 	"current_clamp": {
      "input_type": "current_clamp",
      "module": "IClamp",
      "node_set": "all",
      "amp": 0.120,
      "delay": 500.0,
      "duration": 1000.0
    }
}
```
We can even add multiple injections
```json
"inputs": {
 	"cclamp1": {
      "input_type": "current_clamp",
      "module": "IClamp",
      "node_set": "all",
      "amp": 0.150,
      "delay": 0.0,
      "duration": 400.0
    }
    
    "cclamp2": {
      "input_type": "current_clamp",
      "module": "IClamp",
      "node_set": "all",
      "amp": 0.300,
      "delay": 500.0,
      "duration": 400.0
    }
    
    "cclamp3": {
      "input_type": "current_clamp",
      "module": "IClamp",
      "node_set": "all",
      "amp": 0.450,
      "delay": 1000.0,
      "duration": 400.0
    }
}
```

### Changing cell models
When building the network we defined the cell model and morphology through the 'dynamics_params' and 'morphology_file' options. After building and saving the network, these values were saved in the node-types csv file.

In [ ]:
import pandas as pd
pd.read_csv('network/mcortex_node_types.csv', sep=' ')

If we want to run the simulation on a different cell model, all we have to do is 
1. Download new parameters.json and morphology.swc into components/biophysical
2. Open mcortex_node_types.csv in a text editor and update 'morphology_file' and 'params_file' accordingly.

In our simple one-cell example, it is likely faster to just rebuild the network. However the advantage of the use of the node types becomes clear once we start dealing with a larger network. For example we may have a network of hundreds of thousands of individual cells with tens of thousands of Scnn1a type cells. The process of adjusting/changing the Scnn1a parameter in the csv then starting another simulation only takes seconds, whereas rebuilding the entire network may take hours.